In [1]:
import tensorflow as tf
import numpy as np

In [2]:
xy = np.loadtxt( 'Data/data-04-zoo.csv', delimiter = ',', dtype = np.float32 )
X_data = xy[ :, 0 : -1 ]
N = X_data.shape[0]
y_data = xy[ :, [ -1 ] ]

In [3]:
print( 'y has one of the following values' )
print( np.unique( y_data ) )

y has one of the following values
[0. 1. 2. 3. 4. 5. 6.]


In [4]:
print( 'Shape of X data: ', X_data.shape )
print( 'Shape of y data: ', y_data.shape )

Shape of X data:  (101, 16)
Shape of y data:  (101, 1)


In [5]:
nb_classes = 7

In [6]:
target = tf.one_hot( tf.cast( y_data, tf.int32 ), nb_classes )
target = tf.reshape( target, [ -1, nb_classes ] )
target = tf.cast( target, tf.float32 )

In [7]:
W = tf.Variable( tf.random.normal( [ 16, nb_classes ] ), name = 'weight' )
b = tf.Variable( tf.random.normal( [ nb_classes ] ), name = 'bias' )

In [8]:
def sigma( X ):
    return 1. / ( 1. + tf.exp( -X ) )

def sigma_prime( X ):
    return sigma( X ) * ( 1. - sigma( X ) )

In [9]:
def Hypothesis( X ):
    return ( sigma( tf.matmul( X, W ) + b ) )

def Loss( X, Y ):
    y_pred = Hypothesis( X )
    loss_i = -Y * tf.math.log( y_pred ) - ( 1. - Y ) * tf.math.log( 1. - y_pred )
    return ( tf.reduce_sum( loss_i ) )

In [10]:
def d_loss( X, Y ):
    y_pred = Hypothesis( X )
    
    return ( ( y_pred - Y ) / ( y_pred * ( 1. - y_pred ) + 1e-7 ) )

def d_sigma( X ):
    return ( sigma_prime( tf.matmul( X, W ) + b ) )

d_layer = lambda X, Y: d_loss( X, Y ) * d_sigma( X )
d_b = lambda X, Y: d_layer( X, Y )
d_W = lambda X, Y: tf.matmul( tf.transpose( X ), d_layer( X, Y ) )

In [11]:
learning_rate = 0.01
def Train_step( X, Y ):
    return ( [
        W.assign( W - learning_rate * d_W( X, Y ) ),
        b.assign( b - learning_rate * tf.reduce_sum( d_b( X, Y ) ) )
             ])

prediction = lambda X: tf.argmax( Hypothesis( X ), 1 )

def Acct( X, Y ):
    acct_mat = tf.equal( tf.argmax( Hypothesis( X ), 1 ), tf.argmax( Y, 1 ) )
    
    return ( tf.reduce_mean( tf.cast( acct_mat, tf.float32 ) ) )

In [12]:
for step in range( 500 ):
    Train_step( X_data, target )
    
    if step % 10 == 0:
        step_loss = Loss( X_data, target )
        acc = Acct( X_data, target )
        
        print( 'Step: {:5}\t Loss: {:10.5f}\t Acc: {:.2%}'.format(
            step, step_loss, acc))

Step:     0	 Loss:  355.59796	 Acc: 38.61%
Step:    10	 Loss:  116.58677	 Acc: 84.16%
Step:    20	 Loss:   90.54292	 Acc: 87.13%
Step:    30	 Loss:   76.35056	 Acc: 90.10%
Step:    40	 Loss:   66.74164	 Acc: 92.08%
Step:    50	 Loss:   59.62422	 Acc: 92.08%
Step:    60	 Loss:   54.08026	 Acc: 94.06%
Step:    70	 Loss:   49.61725	 Acc: 95.05%
Step:    80	 Loss:   45.93731	 Acc: 95.05%
Step:    90	 Loss:   42.84523	 Acc: 96.04%
Step:   100	 Loss:   40.20546	 Acc: 97.03%
Step:   110	 Loss:   37.92023	 Acc: 97.03%
Step:   120	 Loss:   35.91748	 Acc: 97.03%
Step:   130	 Loss:   34.14330	 Acc: 97.03%
Step:   140	 Loss:   32.55695	 Acc: 97.03%
Step:   150	 Loss:   31.12724	 Acc: 97.03%
Step:   160	 Loss:   29.82996	 Acc: 97.03%
Step:   170	 Loss:   28.64606	 Acc: 97.03%
Step:   180	 Loss:   27.56029	 Acc: 97.03%
Step:   190	 Loss:   26.56026	 Acc: 97.03%
Step:   200	 Loss:   25.63576	 Acc: 98.02%
Step:   210	 Loss:   24.77825	 Acc: 98.02%
Step:   220	 Loss:   23.98050	 Acc: 100.00%
Step:   23

In [13]:
pred = prediction( X_data )
for p, y in zip( pred, y_data ):
    msg = '[{}]\t Prediction: {:d}\t True y: {:d}'
    print( msg.format( p == int( y[0] ), p, int( y[0] ) ) )

[True]	 Prediction: 0	 True y: 0
[True]	 Prediction: 0	 True y: 0
[True]	 Prediction: 3	 True y: 3
[True]	 Prediction: 0	 True y: 0
[True]	 Prediction: 0	 True y: 0
[True]	 Prediction: 0	 True y: 0
[True]	 Prediction: 0	 True y: 0
[True]	 Prediction: 3	 True y: 3
[True]	 Prediction: 3	 True y: 3
[True]	 Prediction: 0	 True y: 0
[True]	 Prediction: 0	 True y: 0
[True]	 Prediction: 1	 True y: 1
[True]	 Prediction: 3	 True y: 3
[True]	 Prediction: 6	 True y: 6
[True]	 Prediction: 6	 True y: 6
[True]	 Prediction: 6	 True y: 6
[True]	 Prediction: 1	 True y: 1
[True]	 Prediction: 0	 True y: 0
[True]	 Prediction: 3	 True y: 3
[True]	 Prediction: 0	 True y: 0
[True]	 Prediction: 1	 True y: 1
[True]	 Prediction: 1	 True y: 1
[True]	 Prediction: 0	 True y: 0
[True]	 Prediction: 1	 True y: 1
[True]	 Prediction: 5	 True y: 5
[True]	 Prediction: 4	 True y: 4
[True]	 Prediction: 4	 True y: 4
[True]	 Prediction: 0	 True y: 0
[True]	 Prediction: 0	 True y: 0
[True]	 Prediction: 0	 True y: 0
[True]	 Pr